## Part 1: Existing Machine Learning Services

<a href="https://colab.research.google.com/github/peckjon/hosting-ml-as-microservice/blob/master/part1/score_reviews_via_service.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Obtain labelled reviews

In order to test any of the sentiment analysis APIs, we need a labelled dataset of reviews and their sentiment polarity. We'll use NLTK to download the movie_reviews corpus.

In [ ]:
from nltk import download
import boto3
import sys

download('movie_reviews')

### Load the data

The files in movie_reviews have already been divided into two sets: positive ('pos') and negative ('neg'), so we can load the raw text of the reviews into two lists, one for each polarity.

In [ ]:
from nltk.corpus import movie_reviews

# extract words from reviews, pair with label

reviews_pos = []

for fileid in movie_reviews.fileids('pos'):
    review = movie_reviews.raw(fileid)
    reviews_pos.append(review)

reviews_neg = []
for fileid in movie_reviews.fileids('neg'):
    review = movie_reviews.raw(fileid)
    reviews_neg.append(review)


In [ ]:
### Connect to the scoring API

Fill in this function with code that connects to one of these APIs, and uses it to score a single review:

* [Amazon Comprehend: Detect Sentiment](https://docs.aws.amazon.com/comprehend/latest/dg/API_DetectSentiment.html)
* [Google Natural Language: Analyzing Sentiment](https://cloud.google.com/natural-language/docs/analyzing-sentiment)
* [Azure Cognitive Services: Sentiment Analysis](https://docs.microsoft.com/en-us/azure/cognitive-services/text-analytics/how-tos/text-analytics-how-to-sentiment-analysis)
* [Algorithmia: Sentiment Analysis](https://algorithmia.com/algorithms/nlp/SentimentAnalysis)

Your function must return either 'pos' or 'neg', so you'll need to make some decisions about how to map the results of the API call to one of these values. For example, Amazon Comprehend can return "NEUTRAL" or "MIXED" for the Sentiment -- if this happens, you may with to inspect the numeric values under the SentimentScore to see whether it leans toward positive or negative.


In [ ]:

def score_review(review):
    comprehend_client = boto3.client('comprehend')
    response = comprehend_client.detect_sentiment(Text=review,LanguageCode="en")
    response_sentiment = response['Sentiment']
    
    if response_sentiment == "POSITIVE":
        review_sentiment = "pos"
    elif response_sentiment == 'NEGATIVE':
        review_sentiment = 'neg'
    elif response['SentimentScore']['Positive'] > response['SentimentScore']['Negative']:
        review_sentiment = 'pos'
    else:
        review_sentiment = 'neg'
    # TBD: call the service and return 'pos' or 'neg'
    
    return review_sentiment


### Score each review

Now, we can use the function you defined to score each of the reviews.

#### *Note on Testing*

While most of the services listed have free tiers they may be limited to a few thousand requests per week or month, depending on the service. On some platforms you may be billed after reaching that limit. For this reason it is recommended to first test on a smaller set of the reviews, `subset_pos` and `subset_neg`. Once you're happy with your code swap those subsets for the full review sets `reviews_pos` and `reviews_neg`.

In [ ]:
# Create 2 smaller subsets for testing
# Amazon Comprehend only accepts text up to 5000 bytes
filtered_reviews_pos = [ rp for rp in reviews_pos if sys.getsizeof(rp) < 5000 ]
filtered_reviews_neg = [ rp for rp in reviews_neg if sys.getsizeof(rp) < 5000 ]
subset_pos = filtered_reviews_pos
subset_neg = filtered_reviews_neg

results_pos = []
# When comfortable with results switch `subset_pos` to reviews_post`
for review in subset_pos:
    result = score_review(review)
    results_pos.append(result)
    
results_neg = []
# When comfortable with results switch `subset_neg` to reviews_neg`
for review in subset_neg:
    result = score_review(review)
    results_neg.append(result)

### Calculate accuracy

For each of our known positive reviews, we can count the number which our function scored as 'pos', and use this to calculate the % accuracy. We repeaty this for negative reviews, and also for overall accuracy.

In [73]:

correct_pos = results_pos.count('pos')
accuracy_pos = float(correct_pos) / len(results_pos)

correct_neg = results_neg.count('neg')
accuracy_neg = float(correct_neg) / len(results_neg)

correct_all = correct_pos + correct_neg
accuracy_all = float(correct_all) / (len(results_pos)+len(results_neg))
print('Positive reviews: {}% correct'.format(accuracy_pos*100))
print('Negative reviews: {}% correct'.format(accuracy_neg*100))
print('Overall accuracy: {}% correct'.format(accuracy_all*100))

483
Positive reviews: 65.18218623481782% correct
Negative reviews: 69.6470588235294% correct
Overall accuracy: 67.56756756756756% correct
